In [6]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

import math
import openmc
import openmc.mgxs as mgxs
import openmc.data
import random
import h5py
import copy

In [7]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        

In [8]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [3.24]
cs_production_neutrons_h2o = [0.0]

cs_c_value_pu_239 = [1.50]
cs_c_value_h2o = [0.90]

pu_23_cs_1_5 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)


h2o_cs = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, 
                     cs_c_value_pu_239,  cs_total_h2o)

pu_h2o_CS_table_2 = {  
    "pu_23" : pu_23_cs_1_5,
    "h2o" : h2o_cs    
}

energy_groups_2 = [0, 2 * 1E10]

In [9]:
class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

In [10]:
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)

In [11]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [12]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        normal_sq = self.A * self.A + self.B * self.B + self.C * self.C
        
        x_numerator = 2 * self.A * (self.A * angle.tetta_x + self.B * angle.tetta_y +
                                 self.C * angle.tetta_z)
        
        y_numerator = 2 * self.B * (self.A * angle.tetta_x + self.B * angle.tetta_y +
                                 self.C * angle.tetta_z)
        
        z_numerator = 2 * self.C * (self.C * angle.tetta_x + self.B * angle.tetta_y +
                                 self.C * angle.tetta_z)
        
        
        r_reflected = Direction(0, 0)      
        r_reflected.tetta_x = angle.tetta_x - x_numerator / normal_sq
        r_reflected.tetta_y = angle.tetta_y - y_numerator / normal_sq
        r_reflected.tetta_z = angle.tetta_z - z_numerator / normal_sq
        
        return r_reflected
    
    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [13]:
class Cell_Box:
    def __init__(self):
        self.surfaces = []
        self.signs = []
        self.size = 0
        self.x_0 = 0
        self.y_0 = 0
        self.z_0 = 0
        self.x_size = 0
        self.y_size = 0
        self.z_size = 0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
        
    def set_box_sizes(self, x_size, y_size, z_size):
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside  
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size/2. + self.x_0):
            particle.coordinates.x -= self.x_size/2.
        
        if particle.coordinates.y > (self.y_size/2. + self.y_0):
            particle.coordinates.y -= self.y_size/2.
        
        if particle.coordinates.z > (self.z_size/2. + self.z_0):
            particle.coordinates.z -= self.z_size/2.
            
        
        if particle.coordinates.x < (-self.x_size/2. + self.x_0):
            particle.coordinates.x += self.x_size/2.
        
        if particle.coordinates.y < (-self.y_size/2. + self.y_0):
            particle.coordinates.y += self.y_size/2.
        
        if particle.coordinates.z < (-self.z_size/2. + self.z_0):
            particle.coordinates.z += self.z_size/2.
              

In [14]:
x_5 = Plane(0, 1, 0, 10)
x_m5 = Plane(0, 1, 0, -10)

y_5 = Plane(1, 0, 0, 10)
y_m5 = Plane(1, 0, 0, -10)


z_5 = Plane(0, 0, 1, 10)
z_m5 = Plane(0, 0, 1, -10)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]

slab_2 = Cell_Box(surfaces, signs)
slab_2.set_box_sizes(20, 20, 20)
slab_2.set_zero_point(0., 0., 0.)

In [15]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [16]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [17]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [18]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [19]:
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False

In [20]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

In [21]:
def process_real_collision(particle, free_path, universe, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        

In [22]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [23]:
def process_one_particle_history_black_boundary(particle, universe, c_s):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            universe.reflect_particle(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            sum_collisions += process_real_collision(particle, free_path, universe, c_s)
        
            
    return particle, sum_collisions

In [24]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [25]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False

In [26]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    

In [27]:
def splitting_without_random(particles):
     
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)          
            particle.set_weight(current_weight / n_value)
            for j in range(0, n_value - 2):
                particles.append(copy.deepcopy(particle))
            additonal_particle = copy.deepcopy(particle)
            additonal_particle.set_weight(current_weight - n_value)
            particles.append(additonal_particle)
                    

In [28]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [29]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


In [30]:
def flux_estimator(cell_volume, start_weight, particles_current_iteration, cross_section):
    
    sum_weighted_paths = 0.
    
    for i in range(0, len(particles_current_iteration)):
        particle = particles_current_iteration[i]
        energy_group_idx = particle.get_energy_group() 
  
        number_of_production_neutrons = cross_section.number_of_production_neutrons[energy_group_idx]   
        sum_weighted_paths += particles_current_iteration[i].get_weight()* (
            particles_current_iteration[i].get_path())
        
    total_flux = sum_weighted_paths/(cell_volume * start_weight)
    
    return total_flux
    

In [31]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [32]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_current = 0.
        
        for i in range(0, idx - number_inactive):
            std_current += (k_effective[i] - k_effective_exp[idx - number_inactive - 1])**2
            
        std_k_effective.append(math.sqrt(std_current)/(idx * max(idx - 1, 1)))

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]))
            

In [33]:
def calculate_flux(weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]
    flux = sum_collisions / ( sum(weight_previos) * volume * sig_t)

In [34]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        total_collisions = 0.

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history_black_boundary(particle, universe, c_s)
            batch_particles.append(terminate_particle)
            total_collisions += sum_collisions
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std

In [30]:
x_5 = Plane(0, 1, 0, 10)
x_m5 = Plane(0, 1, 0, -10)

y_5 = Plane(1, 0, 0, 10)
y_m5 = Plane(1, 0, 0, -10)


z_5 = Plane(0, 0, 1, 10)
z_m5 = Plane(0, 0, 1, -10)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_2 = Cell_Box(surfaces, signs)
slab_2.set_box_sizes(20, 20, 20)
slab_2.set_zero_point(0., 0., 0.)


In [31]:
test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 100


k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_5)

 num1  3
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
 keff_cycle , k_effective_exp, std_k_effective 2.6072248782962664   2.6072248782962664
i == 102
 keff_cycle , k_effective_exp, std_k_effective 2.616751281313557   2.6119880798049118
i

 keff_cycle , k_effective_exp, std_k_effective 2.609801231719909   2.6137089440682564
i == 180
 keff_cycle , k_effective_exp, std_k_effective 2.601688259464985   2.6135586855107156
i == 181
 keff_cycle , k_effective_exp, std_k_effective 2.619630728556741   2.613633649005111
i == 182
 keff_cycle , k_effective_exp, std_k_effective 2.612994363429153   2.6136258528395504
i == 183
 keff_cycle , k_effective_exp, std_k_effective 2.6041402569194014   2.613511568551356
i == 184
 keff_cycle , k_effective_exp, std_k_effective 2.6149627058430305   2.6135288439953044
i == 185
 keff_cycle , k_effective_exp, std_k_effective 2.614605334328451   2.613541508587459
i == 186
 keff_cycle , k_effective_exp, std_k_effective 2.6094524007042668   2.6134939608213754
i == 187
 keff_cycle , k_effective_exp, std_k_effective 2.6128032549174205   2.613486021673054
i == 188
 keff_cycle , k_effective_exp, std_k_effective 2.6220473790588614   2.6135833098251657
i == 189
 keff_cycle , k_effective_exp, std_k_effective 2.

 keff_cycle , k_effective_exp, std_k_effective 2.6284001852992143   2.6134855013361995
i == 267
 keff_cycle , k_effective_exp, std_k_effective 2.6284079013913737   2.6135748570251525
i == 268
 keff_cycle , k_effective_exp, std_k_effective 2.6169534694218672   2.613594967813228
i == 269
 keff_cycle , k_effective_exp, std_k_effective 2.6166309652712023   2.613612932295228
i == 270
 keff_cycle , k_effective_exp, std_k_effective 2.619963513152843   2.613650288653214
i == 271
 keff_cycle , k_effective_exp, std_k_effective 2.610280522211964   2.6136305824167154
i == 272
 keff_cycle , k_effective_exp, std_k_effective 2.6206909415215405   2.613671631016162
i == 273
 keff_cycle , k_effective_exp, std_k_effective 2.6167249523030223   2.6136892802721556
i == 274
 keff_cycle , k_effective_exp, std_k_effective 2.616837585620517   2.6137073739810543
i == 275
 keff_cycle , k_effective_exp, std_k_effective 2.6183422738336057   2.613733859123069
i == 276
 keff_cycle , k_effective_exp, std_k_effective 2

 keff_cycle , k_effective_exp, std_k_effective 2.6239695434023873   2.6135507091026215
i == 354
 keff_cycle , k_effective_exp, std_k_effective 2.625183944038967   2.6135965092401663
i == 355
 keff_cycle , k_effective_exp, std_k_effective 2.6047957268728146   2.6135619963681376
i == 356
 keff_cycle , k_effective_exp, std_k_effective 2.6095208162470542   2.6135462105082894
i == 357
 keff_cycle , k_effective_exp, std_k_effective 2.618670871476435   2.6135661508233405
i == 358
 keff_cycle , k_effective_exp, std_k_effective 2.6041533769445975   2.6135296672036556
i == 359
 keff_cycle , k_effective_exp, std_k_effective 2.610787671793654   2.613519080348791
i == 360
 keff_cycle , k_effective_exp, std_k_effective 2.62312155328546   2.613556012937009
i == 361
 keff_cycle , k_effective_exp, std_k_effective 2.6130312789610812   2.6135540024620054
i == 362
 keff_cycle , k_effective_exp, std_k_effective 2.609631577795219   2.613539031375491
i == 363
 keff_cycle , k_effective_exp, std_k_effective 2.

 keff_cycle , k_effective_exp, std_k_effective 2.6127522989894665   2.6129949854677297
i == 441
 keff_cycle , k_effective_exp, std_k_effective 2.616933473860771   2.613006535287064
i == 442
 keff_cycle , k_effective_exp, std_k_effective 2.612114888189205   2.6130039281318074
i == 443
 keff_cycle , k_effective_exp, std_k_effective 2.60825384031896   2.6129900794792915
i == 444
 keff_cycle , k_effective_exp, std_k_effective 2.6264161374490116   2.613029108717576
i == 445
 keff_cycle , k_effective_exp, std_k_effective 2.6098549058734584   2.6130199081296217
i == 446
 keff_cycle , k_effective_exp, std_k_effective 2.603647267688146   2.612992819573433
i == 447
 keff_cycle , k_effective_exp, std_k_effective 2.6111335524305943   2.6129874614548654
i == 448
 keff_cycle , k_effective_exp, std_k_effective 2.614996169235985   2.6129932336036616
i == 449
 keff_cycle , k_effective_exp, std_k_effective 2.6304066076277044   2.613043128658172
i == 450
 keff_cycle , k_effective_exp, std_k_effective 2.6

...

 keff_cycle , k_effective_exp, std_k_effective 2.6129888430739   2.6130870646737274
i == 615
 keff_cycle , k_effective_exp, std_k_effective 2.6064381770783718   2.6130741542123777
i == 616
 keff_cycle , k_effective_exp, std_k_effective 2.609062067481258   2.6130663788504958
i == 617
 keff_cycle , k_effective_exp, std_k_effective 2.6085890523693944   2.613057718644536
i == 618
 keff_cycle , k_effective_exp, std_k_effective 2.614814078247681   2.6130611093001406
i == 619
 keff_cycle , k_effective_exp, std_k_effective 2.5992182299642863   2.6130344370856204
i == 620
 keff_cycle , k_effective_exp, std_k_effective 2.605191910697817   2.6130193553041052
i == 621
 keff_cycle , k_effective_exp, std_k_effective 2.6126090978831322   2.6130185678618383
i == 622
 keff_cycle , k_effective_exp, std_k_effective 2.604531661294973   2.6130023094201396
i == 623
 keff_cycle , k_effective_exp, std_k_effective 2.625511180441708   2.6130262269555535
i == 624
 keff_cycle , k_effective_exp, std_k_effective 2.

 keff_cycle , k_effective_exp, std_k_effective 2.6154812425197598   2.612933019810842
i == 702
 keff_cycle , k_effective_exp, std_k_effective 2.6081282550403113   2.612925038474014
i == 703
 keff_cycle , k_effective_exp, std_k_effective 2.607629779739831   2.6129162569504083
i == 704
 keff_cycle , k_effective_exp, std_k_effective 2.627319428601783   2.6129401032610895
i == 705
 keff_cycle , k_effective_exp, std_k_effective 2.6381170167759964   2.612981717994172
i == 706
 keff_cycle , k_effective_exp, std_k_effective 2.6261097754513134   2.613003381455323
i == 707
 keff_cycle , k_effective_exp, std_k_effective 2.6329956840970272   2.613036317703497
i == 708
 keff_cycle , k_effective_exp, std_k_effective 2.6231302349718333   2.613052919541109
i == 709
 keff_cycle , k_effective_exp, std_k_effective 2.608739407908345   2.6130458365991833
i == 710
 keff_cycle , k_effective_exp, std_k_effective 2.59911977002044   2.6130230069818414
i == 711
 keff_cycle , k_effective_exp, std_k_effective 2.61

 keff_cycle , k_effective_exp, std_k_effective 2.615596319754172   2.61303217751593
i == 789
 keff_cycle , k_effective_exp, std_k_effective 2.6244168758080466   2.613048701025788
i == 790
 keff_cycle , k_effective_exp, std_k_effective 2.609032116751901   2.6130428798891594
i == 791
 keff_cycle , k_effective_exp, std_k_effective 2.619684352191378   2.6130524912817816
i == 792
 keff_cycle , k_effective_exp, std_k_effective 2.6239078201284984   2.613068178173179
i == 793
 keff_cycle , k_effective_exp, std_k_effective 2.625655515914291   2.6130863417197028
i == 794
 keff_cycle , k_effective_exp, std_k_effective 2.629804328271094   2.6131104310375
i == 795
 keff_cycle , k_effective_exp, std_k_effective 2.595489951173116   2.613085077829062
i == 796
 keff_cycle , k_effective_exp, std_k_effective 2.615904830978402   2.6130891291985296
i == 797
 keff_cycle , k_effective_exp, std_k_effective 2.599217557532562   2.613069227374045
i == 798
 keff_cycle , k_effective_exp, std_k_effective 2.61268944

 keff_cycle , k_effective_exp, std_k_effective 2.609735011497644   2.613054880941354
i == 876
 keff_cycle , k_effective_exp, std_k_effective 2.6143801168862457   2.6130565887196338
i == 877
 keff_cycle , k_effective_exp, std_k_effective 2.624450790449991   2.6130712530719253
i == 878
 keff_cycle , k_effective_exp, std_k_effective 2.6266521003846126   2.6130887091738697
i == 879
 keff_cycle , k_effective_exp, std_k_effective 2.6152624859346534   2.6130914996446792
i == 880
 keff_cycle , k_effective_exp, std_k_effective 2.614218024093038   2.6130929439067927
i == 881
 keff_cycle , k_effective_exp, std_k_effective 2.613202553176875   2.613093084251569
i == 882
 keff_cycle , k_effective_exp, std_k_effective 2.604552807707174   2.613082163181819
i == 883
 keff_cycle , k_effective_exp, std_k_effective 2.6169043685829254   2.6130870446701984
i == 884
 keff_cycle , k_effective_exp, std_k_effective 2.6240868203585443   2.6131010749963313
i == 885
 keff_cycle , k_effective_exp, std_k_effective 2

 keff_cycle , k_effective_exp, std_k_effective 2.6166209500258817   2.613055019686079
i == 962
 keff_cycle , k_effective_exp, std_k_effective 2.6152565035810134   2.613057573611711
i == 963
 keff_cycle , k_effective_exp, std_k_effective 2.612981547847952   2.6130574855169675
i == 964
 keff_cycle , k_effective_exp, std_k_effective 2.6232448830366986   2.613069276486319
i == 965
 keff_cycle , k_effective_exp, std_k_effective 2.608141728601353   2.6130635798991686
i == 966
 keff_cycle , k_effective_exp, std_k_effective 2.615954086335693   2.613066917666416
i == 967
 keff_cycle , k_effective_exp, std_k_effective 2.6035870721375516   2.613055983588528
i == 968
 keff_cycle , k_effective_exp, std_k_effective 2.6220760378800034   2.613066375356145
i == 969
 keff_cycle , k_effective_exp, std_k_effective 2.6061079887448466   2.613058368006765
i == 970
 keff_cycle , k_effective_exp, std_k_effective 2.6263226182939716   2.6130736142714635
i == 971
 keff_cycle , k_effective_exp, std_k_effective 2.5

In [40]:
def calculate_std(k_effective_exp):
    
    sample_std = []
    for i in range(100, len(k_effective_exp)):
        sample_std.append(np.std(k_effective_exp[:i])/ (i - 1))
        
    return sample_std[-1]
    

In [43]:
2.612903
#2.61448639866804
difference = (k_effective_exp[-1] - 2.612903) * 100000
standart_deviation = calculate_std(k_effective_exp)* 100000


print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  11.822217217138942  with standart deviation [pcm]+- 0.1422076586830413


In [44]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]
cs_production_neutrons_h2o = [0.0]

cs_c_value_pu_239 = [1.50]
cs_c_value_h2o = [0.90]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)


h2o_cs = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, 
                     cs_c_value_pu_239,  cs_total_h2o)

pu_h2o_CS_table_2 = {  
    "pu_23" : pu_23_cs_1_5,
    "h2o" : h2o_cs    
}

energy_groups_2 = [0, 2 * 1E10]

In [45]:
k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4)

 num1  8
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
 keff_cycle , k_effective_exp, std_k_effective 2.297674772715702   2.297674772715702
i == 102
 keff_cycle , k_effective_exp, std_k_effective 2.2918140849046744   2.294744428810188
i =

 keff_cycle , k_effective_exp, std_k_effective 2.2937348380417664   2.2906804030238286
i == 180
 keff_cycle , k_effective_exp, std_k_effective 2.301949650262686   2.290821268614314
i == 181
 keff_cycle , k_effective_exp, std_k_effective 2.273973584096234   2.2906132725091526
i == 182
 keff_cycle , k_effective_exp, std_k_effective 2.2922141930642104   2.2906327959305557
i == 183
 keff_cycle , k_effective_exp, std_k_effective 2.3114282977484755   2.2908833441452296
i == 184
 keff_cycle , k_effective_exp, std_k_effective 2.2868347658247608   2.2908351467842714
i == 185
 keff_cycle , k_effective_exp, std_k_effective 2.3043591221418565   2.2909942523767137
i == 186
 keff_cycle , k_effective_exp, std_k_effective 2.2902487830654144   2.2909855841289075
i == 187
 keff_cycle , k_effective_exp, std_k_effective 2.2790222241358276   2.2908480742439297
i == 188
 keff_cycle , k_effective_exp, std_k_effective 2.272707208130534   2.2906419280380956
i == 189
 keff_cycle , k_effective_exp, std_k_effecti

 keff_cycle , k_effective_exp, std_k_effective 2.2922435257978946   2.290284622725413
i == 267
 keff_cycle , k_effective_exp, std_k_effective 2.2940310083039424   2.2903070561719914
i == 268
 keff_cycle , k_effective_exp, std_k_effective 2.2851857245624827   2.290276572055268
i == 269
 keff_cycle , k_effective_exp, std_k_effective 2.289341391409374   2.2902710384419787
i == 270
 keff_cycle , k_effective_exp, std_k_effective 2.2931310759067602   2.2902878621917715
i == 271
 keff_cycle , k_effective_exp, std_k_effective 2.2823554107542456   2.2902414735868737
i == 272
 keff_cycle , k_effective_exp, std_k_effective 2.2800059339528276   2.2901819646355133
i == 273
 keff_cycle , k_effective_exp, std_k_effective 2.280765203749172   2.2901275324916615
i == 274
 keff_cycle , k_effective_exp, std_k_effective 2.290710536784365   2.290130883091045
i == 275
 keff_cycle , k_effective_exp, std_k_effective 2.2795723502963763   2.2900705486179325
i == 276
 keff_cycle , k_effective_exp, std_k_effective

 keff_cycle , k_effective_exp, std_k_effective 2.2865269611175405   2.2900835147556284
i == 354
 keff_cycle , k_effective_exp, std_k_effective 2.2937952410970963   2.290098127851461
i == 355
 keff_cycle , k_effective_exp, std_k_effective 2.2835390766921186   2.2900724060822086
i == 356
 keff_cycle , k_effective_exp, std_k_effective 2.2879467746048117   2.29006410283425
i == 357
 keff_cycle , k_effective_exp, std_k_effective 2.2891274537530473   2.290060458285296
i == 358
 keff_cycle , k_effective_exp, std_k_effective 2.3002421402842845   2.290099922169013
i == 359
 keff_cycle , k_effective_exp, std_k_effective 2.2875303878522346   2.290090001187095
i == 360
 keff_cycle , k_effective_exp, std_k_effective 2.2939119102335206   2.2901047008372735
i == 361
 keff_cycle , k_effective_exp, std_k_effective 2.2936284750602804   2.2901182018879362
i == 362
 keff_cycle , k_effective_exp, std_k_effective 2.288288609455405   2.290111218710713
i == 363
 keff_cycle , k_effective_exp, std_k_effective 2

 keff_cycle , k_effective_exp, std_k_effective 2.2858860943826276   2.2901872344293404
i == 441
 keff_cycle , k_effective_exp, std_k_effective 2.2941655889639248   2.2901989011581803
i == 442
 keff_cycle , k_effective_exp, std_k_effective 2.2950851396760865   2.2902131884053087
i == 443
 keff_cycle , k_effective_exp, std_k_effective 2.287666151588402   2.2902057626419943
i == 444
 keff_cycle , k_effective_exp, std_k_effective 2.277222032501541   2.290168019240423
i == 445
 keff_cycle , k_effective_exp, std_k_effective 2.299756497704684   2.290195811931624
i == 446
 keff_cycle , k_effective_exp, std_k_effective 2.2948916026776973   2.290209383581179
i == 447
 keff_cycle , k_effective_exp, std_k_effective 2.2990672893857567   2.290234910687244
i == 448
 keff_cycle , k_effective_exp, std_k_effective 2.2861867476810556   2.2902232780349276
i == 449
 keff_cycle , k_effective_exp, std_k_effective 2.292782497153346   2.2902306110409976
i == 450
 keff_cycle , k_effective_exp, std_k_effective 2

...

 keff_cycle , k_effective_exp, std_k_effective 2.2920211582903973   2.290101947593053
i == 615
 keff_cycle , k_effective_exp, std_k_effective 2.292098616044514   2.290105824619172
i == 616
 keff_cycle , k_effective_exp, std_k_effective 2.2915999943499465   2.2901087202969452
i == 617
 keff_cycle , k_effective_exp, std_k_effective 2.2940379850997488   2.290116320422289
i == 618
 keff_cycle , k_effective_exp, std_k_effective 2.2931043741506594   2.2901220888657803
i == 619
 keff_cycle , k_effective_exp, std_k_effective 2.281019104230793   2.2901045493963488
i == 620
 keff_cycle , k_effective_exp, std_k_effective 2.2901107531596083   2.2901045613266624
i == 621
 keff_cycle , k_effective_exp, std_k_effective 2.2919783010030295   2.2901081577559834
i == 622
 keff_cycle , k_effective_exp, std_k_effective 2.290534836947782   2.2901089751490713
i == 623
 keff_cycle , k_effective_exp, std_k_effective 2.284649363283422   2.2900985361206474
i == 624
 keff_cycle , k_effective_exp, std_k_effective 

 keff_cycle , k_effective_exp, std_k_effective 2.2826606517333956   2.290075269359897
i == 702
 keff_cycle , k_effective_exp, std_k_effective 2.2872995775915306   2.2900706585762283
i == 703
 keff_cycle , k_effective_exp, std_k_effective 2.290715110706004   2.2900717273193956
i == 704
 keff_cycle , k_effective_exp, std_k_effective 2.27822200687957   2.2900521085769454
i == 705
 keff_cycle , k_effective_exp, std_k_effective 2.2963266096994657   2.290062479653181
i == 706
 keff_cycle , k_effective_exp, std_k_effective 2.278657176427697   2.290043659020796
i == 707
 keff_cycle , k_effective_exp, std_k_effective 2.281466542027614   2.2900295286797854
i == 708
 keff_cycle , k_effective_exp, std_k_effective 2.2743226355744066   2.29000369497402
i == 709
 keff_cycle , k_effective_exp, std_k_effective 2.2895123712550345   2.290002888202725
i == 710
 keff_cycle , k_effective_exp, std_k_effective 2.2857107266009145   2.28999585187223
i == 711
 keff_cycle , k_effective_exp, std_k_effective 2.2843

 keff_cycle , k_effective_exp, std_k_effective 2.2941656056029465   2.2899730888169416
i == 789
 keff_cycle , k_effective_exp, std_k_effective 2.2811863403705077   2.289960335916439
i == 790
 keff_cycle , k_effective_exp, std_k_effective 2.2847399375676556   2.2899527701217304
i == 791
 keff_cycle , k_effective_exp, std_k_effective 2.289548202061282   2.2899521846397324
i == 792
 keff_cycle , k_effective_exp, std_k_effective 2.2820362422560314   2.2899407454166347
i == 793
 keff_cycle , k_effective_exp, std_k_effective 2.2919902898678433   2.2899437029122356
i == 794
 keff_cycle , k_effective_exp, std_k_effective 2.2943487791347232   2.2899500502843138
i == 795
 keff_cycle , k_effective_exp, std_k_effective 2.297129739451841   2.2899603807723246
i == 796
 keff_cycle , k_effective_exp, std_k_effective 2.2976623665414952   2.289971446843832
i == 797
 keff_cycle , k_effective_exp, std_k_effective 2.307272980748682   2.2899962697045275
i == 798
 keff_cycle , k_effective_exp, std_k_effectiv

 keff_cycle , k_effective_exp, std_k_effective 2.2922630411296607   2.2899788912395747
i == 876
 keff_cycle , k_effective_exp, std_k_effective 2.2856125605847466   2.289973264524813
i == 877
 keff_cycle , k_effective_exp, std_k_effective 2.2856839243345726   2.2899677441384676
i == 878
 keff_cycle , k_effective_exp, std_k_effective 2.29878766033999   2.289979080791683
i == 879
 keff_cycle , k_effective_exp, std_k_effective 2.3031523927933764   2.2899959913334054
i == 880
 keff_cycle , k_effective_exp, std_k_effective 2.288631988328338   2.289994242611604
i == 881
 keff_cycle , k_effective_exp, std_k_effective 2.299201486549596   2.290006031656339
i == 882
 keff_cycle , k_effective_exp, std_k_effective 2.2808390237108807   2.2899943091397845
i == 883
 keff_cycle , k_effective_exp, std_k_effective 2.288690602757008   2.289992644125247
i == 884
 keff_cycle , k_effective_exp, std_k_effective 2.3011743282722033   2.2900069064774757
i == 885
 keff_cycle , k_effective_exp, std_k_effective 2.3

 keff_cycle , k_effective_exp, std_k_effective 2.2856966681460245   2.2900319348160134
i == 963
 keff_cycle , k_effective_exp, std_k_effective 2.2927521767196435   2.290035086892379
i == 964
 keff_cycle , k_effective_exp, std_k_effective 2.293421109192728   2.290039005899671
i == 965
 keff_cycle , k_effective_exp, std_k_effective 2.2769683083234638   2.2900238952666347
i == 966
 keff_cycle , k_effective_exp, std_k_effective 2.288242859935482   2.2900218386438507
i == 967
 keff_cycle , k_effective_exp, std_k_effective 2.3057688021827003   2.290040001231554
i == 968
 keff_cycle , k_effective_exp, std_k_effective 2.3076089701372617   2.290060241979141
i == 969
 keff_cycle , k_effective_exp, std_k_effective 2.298430808848213   2.290069874392109
i == 970
 keff_cycle , k_effective_exp, std_k_effective 2.28116710071052   2.2900596413189116
i == 971
 keff_cycle , k_effective_exp, std_k_effective 2.285607999544164   2.2900545303639466
i == 972
 keff_cycle , k_effective_exp, std_k_effective 2.29

In [46]:
#2.290323
difference = (k_effective_exp[-1] - 2.290323) * 100000
standart_deviation = calculate_std(k_effective_exp)* 100000


print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -19.36324204714346  with standart deviation [pcm]+- 0.053951077981824275


In [47]:
# U235-a

cs_fission_pu_239 = [0.065280]

cs_capture_pu_239 = [0.013056]
cs_scattering_pu_239 = [0.248064]


cs_total_pu_23 = [0.32640]
cs_production_neutrons_pu_239 = [2.70]


pu_23_cs_9_a = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)



In [48]:
test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 50


k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_9_a)

 num1  7
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
 keff_cycle , k_effective_exp, std_k_effective 2.2578138078319134   2.2578138078319134
i == 52
 keff_cycle , k_effective_exp, std_k_effective 2.243148616478996   2.2504812121554547
i == 53
 keff_cycle , k_effective_exp, std_k_effective 2.251721298520827   2.2508945742772455
i == 54
 keff_cycle , k_effective_exp, std_k_effective 2.254718108224394   2.2518504577640326
i == 55
 keff_cycle , k_effective_exp, std_k_effective 2.2500517998389187   2.25149072617901
i == 56
 keff_cycle , k_effective_exp, std_k_effective 2.2500781419584888   2.25125529547559
i == 57
 keff_cycle , k_effect

 keff_cycle , k_effective_exp, std_k_effective 2.255178805901176   2.2509389690632666
i == 135
 keff_cycle , k_effective_exp, std_k_effective 2.2534590269238732   2.250968616802803
i == 136
 keff_cycle , k_effective_exp, std_k_effective 2.2511573690118896   2.250970811595932
i == 137
 keff_cycle , k_effective_exp, std_k_effective 2.260828678323543   2.251084120408893
i == 138
 keff_cycle , k_effective_exp, std_k_effective 2.254302062042226   2.2511206879274535
i == 139
 keff_cycle , k_effective_exp, std_k_effective 2.261358241514027   2.2512357166194374
i == 140
 keff_cycle , k_effective_exp, std_k_effective 2.2425226764112   2.251138905061568
i == 141
 keff_cycle , k_effective_exp, std_k_effective 2.2417099576540522   2.251035290254892
i == 142
 keff_cycle , k_effective_exp, std_k_effective 2.252805114229267   2.2510545274720046
i == 143
 keff_cycle , k_effective_exp, std_k_effective 2.2444484834835103   2.2509834947409457
i == 144
 keff_cycle , k_effective_exp, std_k_effective 2.2448

 keff_cycle , k_effective_exp, std_k_effective 2.2413025936890176   2.2503363763640687
i == 222
 keff_cycle , k_effective_exp, std_k_effective 2.257185198457536   2.2503761950971706
i == 223
 keff_cycle , k_effective_exp, std_k_effective 2.258055180550951   2.2504205822963255
i == 224
 keff_cycle , k_effective_exp, std_k_effective 2.2622292575021605   2.250488448245784
i == 225
 keff_cycle , k_effective_exp, std_k_effective 2.2486491243367697   2.250477937823447
i == 226
 keff_cycle , k_effective_exp, std_k_effective 2.2535917547928643   2.2504956299653185
i == 227
 keff_cycle , k_effective_exp, std_k_effective 2.24037287889347   2.250438439281297
i == 228
 keff_cycle , k_effective_exp, std_k_effective 2.253652653818635   2.250456496666338
i == 229
 keff_cycle , k_effective_exp, std_k_effective 2.235137709489852   2.2503709168497097
i == 230
 keff_cycle , k_effective_exp, std_k_effective 2.239535615228957   2.250310720729594
i == 231
 keff_cycle , k_effective_exp, std_k_effective 2.252

 keff_cycle , k_effective_exp, std_k_effective 2.2478240608039424   2.2500949883564676
i == 309
 keff_cycle , k_effective_exp, std_k_effective 2.246836483194857   2.2500824072554577
i == 310
 keff_cycle , k_effective_exp, std_k_effective 2.265635718379624   2.250142227682858
i == 311
 keff_cycle , k_effective_exp, std_k_effective 2.2523953780281016   2.250150860442802
i == 312
 keff_cycle , k_effective_exp, std_k_effective 2.2505176042869564   2.2501522602284667
i == 313
 keff_cycle , k_effective_exp, std_k_effective 2.2427644309609667   2.250124169622887
i == 314
 keff_cycle , k_effective_exp, std_k_effective 2.24884322328734   2.250119317553434
i == 315
 keff_cycle , k_effective_exp, std_k_effective 2.251404525408684   2.2501241673943975
i == 316
 keff_cycle , k_effective_exp, std_k_effective 2.2533111965328128   2.2501361487069476
i == 317
 keff_cycle , k_effective_exp, std_k_effective 2.251285568239919   2.250140453649019
i == 318
 keff_cycle , k_effective_exp, std_k_effective 2.24

 keff_cycle , k_effective_exp, std_k_effective 2.23571696004728   2.2500897769086143
i == 396
 keff_cycle , k_effective_exp, std_k_effective 2.253179793322159   2.2500987075918903
i == 397
 keff_cycle , k_effective_exp, std_k_effective 2.251787554541909   2.2501035745859825
i == 398
 keff_cycle , k_effective_exp, std_k_effective 2.2631530339946715   2.2501410730325593
i == 399
 keff_cycle , k_effective_exp, std_k_effective 2.2433044989868827   2.250121483995179
i == 400
 keff_cycle , k_effective_exp, std_k_effective 2.244983039979113   2.250106802726562
i == 401
 keff_cycle , k_effective_exp, std_k_effective 2.2573828474729907   2.250127532198774
i == 402
 keff_cycle , k_effective_exp, std_k_effective 2.2495140475444377   2.2501257893446422
i == 403
 keff_cycle , k_effective_exp, std_k_effective 2.244551875105848   2.25010999921932
i == 404
 keff_cycle , k_effective_exp, std_k_effective 2.2386716893908734   2.250077687609635
i == 405
 keff_cycle , k_effective_exp, std_k_effective 2.237

KeyboardInterrupt: 

In [ ]:
difference = (k_effective_exp[-1] - 2.250000) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


In [34]:
# U235-b

cs_fission_pu_239 = [0.065280]

cs_capture_pu_239 = [0.013056]
cs_scattering_pu_239 = [0.248064]


cs_total_pu_23 = [0.32640]
cs_production_neutrons_pu_239 = [2.797101]


pu_23_cs_9_b = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

In [35]:
test_number_of_particles = 30
test_number_interations = 100
test_number_inactive = 50


k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_9_b)

 num1  5
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
 keff_cycle , k_effective_exp, std_k_effective 2.3255792295685764   2.3255792295685764  0.0
i == 52
 keff_cycle , k_effective_exp, std_k_effective 2.3255619692871017   2.325570599427839  4.602135021110564e-09
i == 53
 keff_cycle , k_effective_exp, std_k_effective 2.3485819015274267   2.3332410334610345  6.817363371214686e-06
i == 54
 keff_cycle , k_effective_exp, std_k_effective 2.3123341882412434   2.3280143221560867  9.116990635908888e-06
i == 55
 keff_cycle , k_effective_exp, std_k_effective 2.3302557131175052   2.3284626003483706  8.811356450554932e-06
i == 56
 keff_cycle , 

In [36]:
difference = (k_effective_exp[-1] - 2.330917) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -280.3217425913385  with standart deviation [pcm]+- 0.5915577107362358


In [37]:
# U235-c

cs_fission_pu_239 = [0.065280]

cs_capture_pu_239 = [0.013056]
cs_scattering_pu_239 = [0.248064]


cs_total_pu_23 = [0.32640]
cs_production_neutrons_pu_239 = [2.707308]


pu_23_cs_9_c = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

In [38]:
test_number_of_particles = 20
test_number_interations = 100
test_number_inactive = 50


k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_9_c)

 num1  3
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
 keff_cycle , k_effective_exp, std_k_effective 2.253993650952398   2.253993650952398  0.0
i == 52
 keff_cycle , k_effective_exp, std_k_effective 2.2519534747069176   2.252973562829658  5.439752858201182e-07
i == 53
 keff_cycle , k_effective_exp, std_k_effective 2.2631420454063225   2.2563630570218796  3.0576676751209763e-06
i == 54
 keff_cycle , k_effective_exp, std_k_effective 2.2534130365968053   2.2556255519156108  3.0767604872030535e-06
i == 55
 keff_cycle , k_effective_exp, std_k_effective 2.26516472233305   2.257533385999099  4.128346633815619e-06
i == 56
 keff_cycle , k_e

In [ ]:
difference = (k_effective_exp[-1] - 2.256083) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


In [ ]:
# U235-d

cs_fission_pu_239 = [0.065280]

cs_capture_pu_239 = [0.013056]
cs_scattering_pu_239 = [0.248064]


cs_total_pu_23 = [0.32640]
cs_production_neutrons_pu_239 = [2.679198]


pu_23_cs_9_d = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

In [ ]:
test_number_of_particles = 20
test_number_interations = 100
test_number_inactive = 50


k_effective, k_effective_exp, k_effective_std = simulation_black_boundaries(slab_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_9_d)

In [ ]:
difference = (k_effective_exp[-1] - 2.232667) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))
